# Milvus

In this example, you will set up by creating a simple Llama Index RAG application with a vector store using Milvus. You'll also set up evaluation and logging with TruLens.

Before running, you'll need to install the following
* Docker Compose ([Instructions](https://docs.docker.com/compose/install/))
* Milvus Standalone ([Instructions](https://milvus.io/docs/install_standalone-docker.md))


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/vector-dbs/milvus/milvus_simple.ipynb)

## Setup

### Install dependencies
Let's install some of the dependencies for this notebook if we don't have them already

In [1]:
#! pip install trulens-eval==0.10.0 llama_index==0.8.4 pymilvus==2.3.0 nltk==3.8.1 html2text==2020.1.16

### Add API keys
For this quickstart, you will need Open AI and Huggingface keys

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "..."

### Import from LlamaIndex and TruLens

In [2]:
from llama_index.storage.storage_context import StorageContext
from llama_index.vector_stores import MilvusVectorStore
from llama_index.llms import OpenAI
from llama_index import (
    VectorStoreIndex,
    SimpleWebPageReader,
    LLMPredictor,
    ServiceContext
)

from trulens_eval import TruLlama, Feedback, Tru, feedback
from trulens_eval.feedback import GroundTruthAgreement, Groundedness
tru = Tru()


🦑 Tru initialized with db url sqlite:///default.sqlite .


### First we need to load documents. We can use SimpleWebPageReader

In [3]:
# load documents
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["http://paulgraham.com/worked.html"]
)

### Next we want to create our vector store index

By default, Llama-Index will do this in memory as follows:

In [4]:
index = VectorStoreIndex.from_documents(documents)

Alternatively, we can create the vector store in pinecone

In [6]:
vector_store = MilvusVectorStore(overwrite=True)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

### In either case, we can create our query engine the same way

In [7]:
query_engine = index.as_query_engine()

### Now we can set the engine up for evaluation and tracking

In [8]:
import numpy as np

# Initialize OpenAI-based feedback function collection class:
openai = feedback.OpenAI()

# Define groundedness
grounded = Groundedness(groundedness_provider=openai)
f_groundedness = Feedback(grounded.groundedness_measure).on(
    TruLlama.select_source_nodes().node.text # context
).on_output().aggregate(grounded.grounded_statements_aggregator)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on_input_output()

# Question/statement relevance between question and each context chunk.
f_qs_relevance = Feedback(openai.qs_relevance).on_input().on(
    TruLlama.select_source_nodes().node.text
).aggregate(np.mean)

✅ In groundedness_measure, input source will be set to *.__record__.app.query.rets.source_nodes[:].node.text .
✅ In groundedness_measure, input statement will be set to *.__record__.main_output or `Select.RecordOutput` .
✅ In relevance, input prompt will be set to *.__record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to *.__record__.main_output or `Select.RecordOutput` .
✅ In qs_relevance, input question will be set to *.__record__.main_input or `Select.RecordInput` .
✅ In qs_relevance, input statement will be set to *.__record__.app.query.rets.source_nodes[:].node.text .


### Instrument query engine for logging with TruLens

In [9]:
tru_query_engine = TruLlama(query_engine,
    app_id='LlamaIndex_App1',
    feedbacks=[f_groundedness, f_qa_relevance, f_qs_relevance])

In [10]:
# Instrumented query engine can operate like the original:
llm_response = tru_query_engine.query("What did the author do growing up?")

print(llm_response)

The author worked on writing and programming outside of school before college. They wrote short stories and tried writing programs on an IBM 1401 computer. They also mentioned later getting a microcomputer and writing simple games, a program to predict rocket heights, and a word processor.


### Explore in a Dashboard

In [11]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.1.157:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

Alternatively, you can run `trulens-eval` from a command line in the same folder to start the dashboard.

### Or view results directly in your notebook

In [12]:
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,latency,total_tokens,total_cost
0,LlamaIndex_App1,"{""app_id"": ""LlamaIndex_App1"", ""tags"": ""-"", ""me...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_521ba4d87ce56263ef3a3389ca9427c5,"""What did the author do growing up?""","""The author worked on writing and programming ...",-,"{""record_id"": ""record_hash_521ba4d87ce56263ef3...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2023-08-28T11:52:22.663780"", ""...",2023-08-28T11:52:25.251033,2,2083,0.003139
